In [9]:
import os
import plotly
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
import collections
import re
from operator import itemgetter

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

plotly.tools.set_credentials_file(username='Fernet', api_key='zC7ogVfVT5UPcpu2VKse')
plotly.offline.init_notebook_mode(connected=True)

#---------------- Settings ----------------

image_ext = "svg" #Extension para bajar los graficos
poda = 5 #Cuantos datos se dejan a cada lado al podar
#image_h = 1000
#image_w = 1400
image_h = 600
image_w = 800

#---------------- Funciones ----------------

#Exp 1.1
def procesar_archivo_ej1_exp1(data_path):
    df = pd.read_csv(data_path, sep = ";")
    cant_exp = df.shape[0] #Shape = (filas, columnas)
    proms = []
    #Ordeno las instancias por tiempo
    df.sort_values("Tiempo", inplace = "True")
    #Descarto los valores en los extremos
    df = df.iloc[poda : cant_exp - poda, :]

    promedio = df["Tiempo"].mean() #Promedio podado de las instancias
    
    return promedio

def procesar_carpeta_ej1_exp1(folder_path):
    #Settings
    cant_caminos = [5, 10, 15, 20, 25, 30, 35, 40, 50, 75, 80, 95, 100]
    buenosCasos = []
    malosCasos = []
    largo_camino = 20
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".csv"):
                file = folder_path + "/" + file
                f = open(file, 'r')
                prom = procesar_archivo_ej1_exp1(file)
                caminos_exp = [int(s) for s in re.split('[_ .]', file)  if s.isdigit()]
                if "buenCaso" in file:
                    buenosCasos.append((prom, caminos_exp))
                else:
                    malosCasos.append((prom, caminos_exp))
                f.close()
    
    buenosCasos = sorted(buenosCasos, key = itemgetter(1))
    malosCasos = sorted(malosCasos, key = itemgetter(1))
    
    return cant_caminos, largo_camino, buenosCasos, malosCasos

#Exp 1.2
def procesar_archivo_ej1_exp2(data_path): # NO SE USA
    df = pd.read_csv(data_path, sep = ";")
    cant_instancias = df.shape[0] #Shape = (filas, columnas)
    #Cuanto cantidad de veces que no encontro solucion
    sin_solucion = df["Sol"].tolist().count(-1)
    print sin_solucion
    con_solucion = cant_instancias - sin_solucion
    
    return sin_solucion, con_solucion

def procesar_carpeta_ej1_exp2(folder_path):
    #Settings
    casos = []
    promedios = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".csv"):
                file = folder_path + "/" + file
                f = open(file, 'r')
                prom = procesar_archivo_ej1_exp1(file)
                datos_exp = tuple([int(s) for s in re.split('[_ .]', file)  if s.isdigit()])
                casos.append(datos_exp)
                promedios.append(prom)
                f.close()
    
    ordenados = zip(casos, promedios)
    ordenados.sort()
    c, p = zip(*ordenados)
    return c, p

def crearFig_ej1_exp2(ej1_exp2_casos, ej1_exp2_proms, fig_title):
    traces = []
    grupos = []
    for tup, prom in zip(ej1_exp2_casos, ej1_exp2_proms):
        grupo = str(tup[0])
        grupos.append(grupo)
        k = str(tup[1])
        trace = go.Bar(
            x = [int(grupo)],
            y = [prom],
            name = grupo + "_" + k,
            legendgroup = grupo
        )
        traces.append(trace)
    
    layout = go.Layout(
        title = fig_title,
        barmode = 'group',
        xaxis = dict(
            title = "Cantidad de caminos"
        ),
        yaxis = dict(
            title = "Tiempo (ns)"
        )
    )
    return go.Figure(data = traces, layout = layout)

#Experimentos 2.1 y 3.1, n variable
def procesar_archivo_tiempoVsTam(data_path):
    df = pd.read_csv(data_path, sep = ";")
    cant_exp = df.shape[0] #Shape = (filas, columnas)
    proms = []
    #Ordeno las instancias por tiempo
    df.sort_values("Tiempo", inplace = "True")
    #Descarto los valores en los extremos
    df = df.iloc[poda : cant_exp - poda, :]

    promedio = df["Tiempo"].mean() #Promedio podado de las instancias
    varianza = df["Tiempo"].var()  #Varianza de los tiempos
    v = df["V"].iloc[0]    
    
    return v, promedio, varianza

def procesar_carpeta_tiempoVsTam(folder_path):
    #Settings
    tams = []
    promedios = []
    varianzas = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".csv"):
                file = folder_path + "/" + file
                f = open(file, 'r')
                tam, prom, var = procesar_archivo_tiempoVsTam(file)
                tams.append(tam)
                promedios.append(prom)
                varianzas.append(var)
                f.close()
    
    ordenados = zip(tams, promedios, varianzas)
    ordenados.sort()
    t, p, v = zip(*ordenados)
    return t, p, v

#Exp 2.2
def procesar_archivo_ej2_exp2(data_path):
    df = pd.read_csv(data_path, sep = ";")
    cant_exp = df.shape[0] #Shape = (filas, columnas)
    proms = []
    #Ordeno las instancias por tiempo
    df.sort_values("Tiempo", inplace = "True")
    #Descarto los valores en los extremos
    df = df.iloc[poda : cant_exp - poda, :]

    promedio = df["Tiempo"].mean() #Promedio podado de las instancias
    varianza = df["Tiempo"].var()  #Varianza de los tiempos
    v = df["V"].iloc[0]    
    maxPeaje = df["MaxPeaje"].iloc[0]
    
    return v, maxPeaje, promedio, varianza

def procesar_carpeta_ej2_exp2(folder_path):
    #Settings
    maxPeajes = []
    promedios = []
    varianzas = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".csv"):
                file = folder_path + "/" + file
                f = open(file, 'r')
                tam, maxPeaje, prom, var = procesar_archivo_ej2_exp2(file)
                maxPeajes.append(maxPeaje)
                promedios.append(prom)
                varianzas.append(var)
                f.close()
    
    ordenados = zip(maxPeajes, promedios, varianzas)
    ordenados.sort()
    mp, p, v = zip(*ordenados)
    return tam, mp, p, v

#Exp 3.2
def crearFig_ej3_exp2(folder_path, xtitle, ytitle, fig_title):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".csv"):
                file = folder_path + "/" + file
                f = open(file, 'r')
                print "Antes de if"
                if("AGM" in re.split('_', file)):
                    agm_v, agm_prom, agm_var = procesar_archivo_tiempoVsTam(file)
                else:
                    kn_v, kn_prom, kn_var = procesar_archivo_tiempoVsTam(file)
                f.close()
    
    traces = []
    grupos = []
    agm_trace = go.Bar(
            x = ["AGM_" + str(int(agm_v))],
            y = [agm_prom],
            name = "AGM_" + str(int(agm_v)),
            width = [0.8]
    )
    kn_trace = go.Bar(
            x = ["K_" + str(int(kn_v))],
            y = [kn_prom],
            name = "K_" + str(int(kn_v)),
            width = [0.8]
    )
    traces.append(agm_trace)
    traces.append(kn_trace)
    
    layout = go.Layout(
        title = fig_title,
        bargap = 0.5,
        
        xaxis = dict(
            title = xtitle
        ),
        yaxis = dict(
            title = ytitle
        )
    )
    return go.Figure(data = traces, layout = layout)

#Toma un vector de tuplas (x, y, nombre) y devuelve una figura con cada tupla de datos
def crear_fig(traces, xtitle, ytitle, fig_title):
    data = []
    for trace in traces:
        #Para que todas las traces sean distintas ademas del color
        i = 0
        modo = str(i)
        i += 1

        new_trace = go.Scatter(
            x = trace[0],
            y = trace[1],
            name = trace[2],
            mode = modo
                
        )
        data.append(new_trace)
    
    layout = go.Layout(
        xaxis=dict(
            tickmode = "auto",
            #nticks = cant_exp,
            ticks = "inside",
            #tick0 = 0,
            #ticklen = 8,
            #dtick = 3,
            title = xtitle,
            tickangle = 0
        ),
        yaxis=dict(
            tick0 = 0,
            zeroline = "False",
            ticks='inside',
            ticklen=8,
            tickwidth=3,
            tickcolor='#000',
            title = ytitle
        ),
        legend=dict(
            x=10,
            y=1,
            traceorder='normal',
            font=dict(
                family='sans-serif',
                size=14,
                color='#000'
            ),
            bgcolor='#E2E2E2',
            bordercolor='#FFFFFF',
            borderwidth=2
        ),
        title = fig_title
    )
    
    return go.Figure(data=data, layout=layout)

#---------------- Script ----------------

#Procesamiento de datos

data_ej1_exp1 = procesar_carpeta_ej1_exp1("Outputs/Ej1/outsTiempo/exp1")
ej1_exp1_cantCaminos =  data_ej1_exp1[0]
ej1_exp1_largoCaminos = data_ej1_exp1[1]
ej1_exp1_buenosCasos =  [tiempo[0] for tiempo in data_ej1_exp1[2]]
ej1_exp1_malosCasos =  [tiempo[0] for tiempo in data_ej1_exp1[3]]


data_ej1_exp2 = procesar_carpeta_ej1_exp2("Outputs/Ej1/outsTiempo/exp2")
ej1_exp2_casos = data_ej1_exp2[0] # Vector de  tuplas (cant_caminos, k)
ej1_exp2_proms = data_ej1_exp2[1]

data_ej2_exp1 = procesar_carpeta_tiempoVsTam("Outputs/Ej2/outsTiempo/exp1")
ej2_exp1_tams =  data_ej2_exp1[0]
ej2_exp1_proms =  data_ej2_exp1[1]
ej2_exp1_vars =  data_ej2_exp1[2]
#ej2_exp1_vars

data_ej2_exp2 = procesar_carpeta_ej2_exp2("Outputs/Ej2/outsTiempo/exp2")
ej2_exp2_tam =  data_ej2_exp2[0]
ej2_exp2_maxPeajes =  data_ej2_exp2[1]
ej2_exp2_proms =  data_ej2_exp2[2]
ej2_exp2_vars =  data_ej2_exp2[3]

data_ej3_exp1 = procesar_carpeta_tiempoVsTam("Outputs/Ej3/outsTiempo/exp1")
ej3_exp1_tams =  data_ej3_exp1[0]
ej3_exp1_proms =  data_ej3_exp1[1]
ej3_exp1_vars =  data_ej3_exp1[2]

#Traces
ejer1_exp1_buenosCasos_trace = (ej1_exp1_cantCaminos, ej1_exp1_buenosCasos, "Casos favorables")
ejer1_exp1_malosCasos_trace = (ej1_exp1_cantCaminos, ej1_exp1_malosCasos, "Casos desfavorables")

ej2_exp1_trace = (ej2_exp1_tams, ej2_exp1_proms, "ejer2_exp1")
ej2_exp2_trace = (ej2_exp2_maxPeajes, ej2_exp2_proms, "ejer2_exp2")

ej3_exp1_trace = (ej3_exp1_tams, ej3_exp1_proms, "ejer3_exp1")


#Graficos

#Exp 1.1
nombre = "Ejercicio 1, experimento 1. <br>Largo de los caminos = %d" %ej1_exp1_largoCaminos

traces = [ejer1_exp1_buenosCasos_trace, ejer1_exp1_malosCasos_trace]

fig = crear_fig(traces, "Cantidad de caminos", "Tiempo promedio (ns)", nombre)

#plotly.offline.iplot(fig)
#plotly.offline.iplot(fig, image = image_ext, image_width = image_w, image_height = image_h, filename = "Ej1Exp1Scatter")

#Exp 1.2
nombre = "Ejercicio 1, experimento 2. <br>Largo de los caminos = cantidad de caminos - 1"

fig = crearFig_ej1_exp2(ej1_exp2_casos, ej1_exp2_proms, nombre)

#plotly.offline.iplot(fig)
plotly.offline.iplot(fig, image = image_ext, image_width = image_w, image_height = image_h, filename = "Ej1Exp2Barras")

#Exp 2.1
nombre = "Ejercicio 2, experimento 1."

traces = [ej2_exp1_trace]

fig = crear_fig(traces, "Cantidad de vertices", "Tiempo promedio (ns)", nombre)

#plotly.offline.iplot(fig)
#plotly.offline.iplot(fig, image = image_ext, image_width = image_w, image_height = image_h, filename = "Ej2Exp1Scatter")

#Exp 2.2
nombre = "Ejercicio 2, experimento 2.<br> n = %d" %ej2_exp2_tam

traces = [ej2_exp2_trace]

fig = crear_fig(traces, "Costo del peaje más caro", "Tiempo promedio (ns)", nombre)

#plotly.offline.iplot(fig)
#plotly.offline.iplot(fig, image = image_ext, image_width = image_w, image_height = image_h, filename = "Ej2Exp2Scatter")

#Exp 3.1
nombre = "Ejercicio 3, experimento 1."

traces = [ej3_exp1_trace]

fig = crear_fig(traces, "Cantidad de vertices", "Tiempo promedio (ns)", nombre)

#plotly.offline.iplot(fig)
#plotly.offline.iplot(fig, image = image_ext, image_width = image_w, image_height = image_h, filename = "Ej3Exp1Scatter")

#Exp 3.2
nombre = "Ejercicio 3, experimento 2."

fig = crearFig_ej3_exp2("Outputs/Ej3/outsTiempo/exp2", "Tipo de grafo", "Tiempo (ns)", nombre)

#plotly.offline.iplot(fig)
#plotly.offline.iplot(fig, image = image_ext, image_width = image_w, image_height = image_h, filename = "Ej3Exp2Barras")



Antes de if
Antes de if
